# Building Knowledge Graph with GLiNER

In [ ]:
!pip install -U -q gliner torch openai neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import json

import torch
print(torch.__version__)

from gliner import GLiNER
from openai import OpenAI
from neo4j import GraphDatabase

2.4.1+cu121


In [ ]:
model = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

gliner_multitask_performance.png:   0%|          | 0.00/76.8k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
client = OpenAI(api_key='foobar')

## Implement Helper Functions

In [ ]:
def enhance_text(text: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": f"""
            ## CONTEXT
            {original_text}

            ## INSTRUCTION
            Rangkum teks di atas dan sebutkan data penting.
            Keluarkan dalam json.

            ## EXAMPLE
            Context:
            # Analisis Numerik

            # Informasi Umum

            |Kode Kuliah|CSCM603117|
            |---|---|
            |Kredit SKS|3|
            |Prasyarat|CSGE602012 Aljabar Linier CSCM601213 Kalkulus 2|
            |Metode Evaluasi|Ujian, Tugas|
            |Workshop/Lab.|Latihan dengan pemrograman|
            |CPL Prodi|CPL 6 Prodi IK|

            # Deskripsi Kuliah

            Mata kuliah ini membahas dasar-dasar penerapan komputasi ilmiah untuk ilmu komputer. Topik yang dibahas meliputi prinsip-prinsip matematika dasar serta algoritma analisis numerik sampai pada isu-isu praktis seperti reliabilitas perangkat lunak dan kinerja pada perangkat keras mutakhir; dan juga teknik optimisasi yang relevan dengan isu machine learning yang populer akhir-akhir. Peserta dilatih untuk memecahkan persoalan terkait tema-tema analisis numerik. Beberapa latihan dibuat dengan pembuatan program dengan bahasa apapun, namun dianjurkan C, Python, Octave atau tools open source lainnya.

            # Topik

            - (1) Introduction: computation in finite precision: machine representation numbers, errors propagation and analysis, numerical stability and accuracy
            - (2) System of linear equations: review relevant theory of linear algebra, triangular factorization, pivoting strategies
            - (3) System of linear equations: special linear system
            - (4) Least Squares Problems
            - (5) Nonlinear equation
            - (6) Optimization
            - (7) Interpolation
            - (8) Numerical Integration
            - (9) Initial value problems in ordinary differential equations

            # Capaian Pembelajaran Mata Kuliah (CPMK)

            Mampu implementasi sistem yang memanfaatkan metode numerik untuk menyelesaikan masalah yang membutuhkan pemodelan matematika yang kompleks.

            # Sumber Daya Pembelajaran

            Scientific Computing - An introductory survey, 2nd Ed, McGraw -Hill, Michael T. Heath, 2002

            Response:
            {{
                "KodeKuliah": "CSCM603117",
                "NamaKuliah": "Analisis Numerik",
                "KreditSKS": 3,
                "Prasyarat": [
                    "CSGE602012 Aljabar Linier",
                    "CSCM601213 Kalkulus 2"
                ],
                "MetodeEvaluasi": [
                    "Ujian",
                    "Tugas"
                ],
                "WorkshopLab": "Latihan dengan pemrograman",
                "CPLProdi": "CPL 6 Prodi IK",
                "Deskripsi": "Mata kuliah ini membahas dasar-dasar penerapan komputasi ilmiah untuk ilmu komputer. Topik yang dibahas meliputi prinsip-prinsip matematika dasar serta algoritma analisis numerik sampai pada isu-isu praktis seperti reliabilitas perangkat lunak dan kinerja pada perangkat keras mutakhir; dan juga teknik optimisasi yang relevan dengan isu machine learning yang populer akhir-akhir. Peserta dilatih untuk memecahkan persoalan terkait tema-tema analisis numerik. Beberapa latihan dibuat dengan pembuatan program dengan bahasa apapun, namun dianjurkan C, Python, Octave atau tools open source lainnya.",
                "Topik": [
                    "Introduction: computation in finite precision: machine representation numbers, errors propagation and analysis, numerical stability and accuracy",
                    "System of linear equations: review relevant theory of linear algebra, triangular factorization, pivoting strategies",
                    "System of linear equations: special linear system",
                    "Least Squares Problems",
                    "Nonlinear equation",
                    "Optimization",
                    "Interpolation",
                    "Numerical Integration",
                    "Initial value problems in ordinary differential equations"
                ],
                "CPMK": "Mampu implementasi sistem yang memanfaatkan metode numerik untuk menyelesaikan masalah yang membutuhkan pemodelan matematika yang kompleks.",
                "SumberDayaPembelajaran": {{
                    "Judul": "Scientific Computing - An introductory survey",
                    "Edisi": "2nd Ed",
                    "Penerbit": "McGraw-Hill",
                    "Penulis": "Michael T. Heath",
                    "Tahun": 2002
                }}
            }}
            """}
        ]
    )

    improved_text = response.choices[0].message.content.strip('```json').strip('```').strip()

    data = json.loads(improved_text)

    return improved_text

In [ ]:
def extract_relation(text: str, course_name: str) -> list:
    classes = ["mata kuliah", "bobot sks", "prasyarat", "semester", "dosen", "kategori mata kuliah", "kode mata kuliah", "istilah", "deskripsi", "evaluasi", "lab", "topik"]

    entities = model.predict_entities(text, classes)

    entity_category = {}

    for entity in entities:
        entity_category[entity['text']] = entity['label']

    labels = [
        f"{course_name} <> deskripsi",
        f"{course_name} <> bobot SKS",
        f"{course_name} <> kode mata kuliah",
        f"{course_name} <> metode evaluasi",
        f"{course_name} <> lab",
        f"{course_name} <> topik",
        f"{course_name} <> cpmk",
        f"{course_name} <> prasyarat",
        f"{course_name} <> sumber daya pembelajaran",
    ]

    entities = model.predict_entities(text, labels)

    relations = []

    for entity in entities:
        head = entity['label'].split(' <> ')[0]
        relation = entity['label'].split(' <> ')[1]
        tail = entity['text']
        if relation in ['prasyarat', 'sumber daya pembelajaran']:
            continue
        relations.append({
            'head': head,
            'type': relation,
            'tail': tail,
        })

    return relations

In [ ]:
original_text = """
# Aljabar Linier

# Informasi Umum

|Kode Kuliah|CSGE602012|
|---|---|
|Kredit SKS|3|
|Prasyarat|-|
|Metode Evaluasi|Ujian, Diskusi, Tugas|
|Workshop/Lab.|-|
|CPL Prodi|CPL 6 Prodi IK|

# Deskripsi Kuliah

Mata ajar Aljabar Linear membekali mahasiswa keterampilan pemecahan masalah terkait dengan aljabar vektor dan meningkatkan penalaran matematis (logis, konsisten, jelas, seksama) yang merupakan disposisi dan keterampilan berpikir kritis. Bahan kajian meliputi sistem persamaan linear, aljabar matriks, determinan, ruang vektor dan hasil kali dalam, nilai dan vektor eigen, serta transformasi linear.

# Topik

Linear equation systems, matrices, determinant, vector spaces, inner product spaces, Eigen value and Eigen vector, and linear transformation.

# Capaian Pembelajaran Mata Kuliah (CPMK)

1. Mahasiswa mampu menerapkan operasi aritmetika matriks dan determinan, menggunakan metode-metode penyelesaian sistem persamaan linear, serta dapat menentukan dan menerapkan metode penyelesaian sistem persamaan linear yang efektif;
2. Mahasiswa mampu menjelaskan ruang vektor umum, dapat menentukan basis dan dimensi ruang vektor umum, dan mengonstruksi subruang berdasarkan matriks yang diberikan;
3. Mahasiswa mampu mengonstruksi ruang hasil kali dalam, menentukan panjang, jarak, sudut, dan ortogonalitas komplemen, serta dapat menerapkannya dalam menentukan metode hampiran kuadrat terkecil;
4. Mahasiswa mampu menentukan nilai eigen, dan ruang eigen matriks persegi serta dapat menerapkannya untuk mendiagonalisasi matriks dan menentukan basis R^n yang terdiri atas vektor-vektor eigen;
5. Mahasiswa mampu mengidentifikasi apakah suatu fungsi merupakan transformasi linear, menentukan matriks standar, rank, nulitas, nilai dan vektor eigen transformasi
# linear di ruang Euclid, serta dapat menginterpretasikan secara geometris transformasi linear pada bidang dan ruang.

# Sumber Daya Pembelajaran

|1.|Anton, Howard; Elementary Linear Algebra; 11th Edition, John Wiley & Sons. Inc, New York, NY, 2013.|
|---|---|
|2.|Lay, David C.; Linear Algebra and Its Application; 2nd Edition, Addison-Wesley Publ. Co.; Reading, Mass, 2000|
|3.|Johnson, Lee W., R. Dean Riess, Jimmy T. Arnold; Introduction to Linear Algebra, Addison Wesley, New York, NY, 2002|
"""

improved_text = enhance_text(original_text)
print(improved_text)

relations = extract_relation(improved_text, 'Aljabar Linier')
print(relations)

{
    "KodeKuliah": "CSGE602012",
    "NamaKuliah": "Aljabar Linier",
    "KreditSKS": 3,
    "Prasyarat": "-",
    "MetodeEvaluasi": [
        "Ujian",
        "Diskusi",
        "Tugas"
    ],
    "WorkshopLab": "-",
    "CPLProdi": "CPL 6 Prodi IK",
    "Deskripsi": "Mata ajar Aljabar Linear membekali mahasiswa keterampilan pemecahan masalah terkait dengan aljabar vektor dan meningkatkan penalaran matematis (logis, konsisten, jelas, seksama) yang merupakan disposisi dan keterampilan berpikir kritis. Bahan kajian meliputi sistem persamaan linear, aljabar matriks, determinan, ruang vektor dan hasil kali dalam, nilai dan vektor eigen, serta transformasi linear.",
    "Topik": [
        "Linear equation systems",
        "Matrices",
        "Determinant",
        "Vector spaces",
        "Inner product spaces",
        "Eigen value and Eigen vector",
        "Linear transformation"
    ],
    "CPMK": [
        "Mahasiswa mampu menerapkan operasi aritmetika matriks dan determinan, menggu

## Implement Knowledge Base

In [ ]:
class KnowledgeBase():
    def __init__(self):
        self.entities = []
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ['head', 'type', 'tail'])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_entity(self, e):
        if e not in self.entities:
            self.entities.append(e)

    def add_relation(self, r):
        self.add_entity(r['head'])
        self.add_entity(r['tail'])

        if not self.exists_relation(r):
            self.relations.append(r)

    def print(self):
        print('Entities:')
        for e in self.entities:
            print(f'  {e}')
        print('Relations:')
        for r in self.relations:
            print(f'  {r}')

    def export_txt(self, filename):
        with open(filename, 'w') as file:
            for relation in self.relations:
                head = relation['head']
                tail = relation['tail']
                relation_type = relation['type']
                file.write(f"{(head, relation_type, tail)}\n")

    def import_txt(self, filename):
        with open(filename, 'r') as file:
            for line in file:
                match = re.match(r"\('(.+?)',\s*'(.+?)',\s*'(.+?)'\)", line.strip())
                if match:
                    head, relation_type, tail = match.groups()
                    self.add_relation({
                        'head': head,
                        'type': relation_type,
                        'tail': tail
                    })

    def persist_to_neo4j(self, uri, username, password):
        driver = GraphDatabase.driver(uri, auth=(username, password))

        with driver.session() as session:
            for entity in self.entities:
                session.run(
                    "MERGE (e:Entity {name: $name})",
                    name=entity
                )

            for relation in self.relations:
                session.run(
                    """
                    MATCH (head:Entity {name: $head})
                    MATCH (tail:Entity {name: $tail})
                    MERGE (head)-[r:RELATION {type: $type}]->(tail)
                    """,
                    head=relation['head'],
                    tail=relation['tail'],
                    type=relation['type']
                )

        driver.close()

In [ ]:
kb = KnowledgeBase()

for relation in relations:
    kb.add_relation(relation)

kb.export_txt("triples.txt")

kb.print()

Entities:
  Aljabar Linier
  CSGE602012
  Ujian
  Mata ajar Aljabar Linear
  Linear equation systems
  Matrices
  Determinant
  Vector spaces
  Inner product spaces
  Eigen value and Eigen vector
  Linear transformation
Relations:
  {'head': 'Aljabar Linier', 'type': 'kode mata kuliah', 'tail': 'CSGE602012'}
  {'head': 'Aljabar Linier', 'type': 'metode evaluasi', 'tail': 'Ujian'}
  {'head': 'Aljabar Linier', 'type': 'deskripsi', 'tail': 'Mata ajar Aljabar Linear'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Linear equation systems'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Matrices'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Determinant'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Vector spaces'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Inner product spaces'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Eigen value and Eigen vector'}
  {'head': 'Aljabar Linier', 'type': 'topik', 'tail': 'Linear transformation'}


## Build Knowledge Graph

In [ ]:
!unzip matkul.zip -d ./

Archive:  matkul.zip
   creating: ./matkul/
  inflating: ./matkul/tkik.txt       
  inflating: ./matkul/pbm.txt        
  inflating: ./matkul/sister.txt     
  inflating: ./matkul/robotika.txt   
  inflating: ./matkul/kp.txt         
  inflating: ./matkul/kg.txt         
  inflating: ./matkul/simmod.txt     
  inflating: ./matkul/tbi.txt        
  inflating: ./matkul/pok.txt        
  inflating: ./matkul/pbl.txt        
  inflating: ./matkul/geokom.txt     
  inflating: ./matkul/ta-kelompok.txt  
  inflating: ./matkul/pemlog.txt     
  inflating: ./matkul/pbk.txt        
  inflating: ./matkul/adsis.txt      
  inflating: ./matkul/tekkom.txt     
  inflating: ./matkul/eh.txt         
  inflating: ./matkul/dl.txt         
  inflating: ./matkul/fortal.txt     
  inflating: ./matkul/komas.txt      
  inflating: ./matkul/psd.txt        
  inflating: ./matkul/bioinformatika.txt  
  inflating: ./matkul/jarkom.txt     
  inflating: ./matkul/mpk-inggris.txt  
  inflating: ./matkul/praktikum-cp.

In [ ]:
kb = KnowledgeBase()

try:
    kb.import_txt('kg.txt')
except:
    pass

In [ ]:
done = []
with open('done.txt', 'r') as file:
    for line in file:
        done.append(line.strip())

for root, dirs, files in os.walk('matkul/'):
    for filename in files:
        if filename.endswith(".txt"):
            input_file = os.path.join(root, filename)

            if filename in done:
                print(f"SKIP: {filename} already processed")
                continue

            print(f"START: Extracting triplets for {filename}")

            with open(input_file, 'r') as file:
                first_line = file.readline()
                match = re.search(r'#\s*(.+)(?:\s+Informasi Umum)?', first_line)
                course_name = match.group(1).strip()

                original_text = file.read()

            improved_text = enhance_text(original_text)
            relations = extract_relation(improved_text, course_name)

            for relation in relations:
                kb.add_relation(relation)

            kb.export_txt("kg.txt")

            with open('done.txt', 'a') as file:
                file.write(f"{filename}\n")

            print(f"DONE")

SKIP: mpk-inggris.txt already processed
SKIP: projut.txt already processed
SKIP: tekkom.txt already processed
SKIP: simmod.txt already processed
SKIP: eh.txt already processed
SKIP: adsis.txt already processed
SKIP: fortal.txt already processed
SKIP: mpkt.txt already processed
SKIP: pbm.txt already processed
SKIP: geokom.txt already processed
SKIP: pendat.txt already processed
SKIP: persdif.txt already processed
SKIP: pbp.txt already processed
SKIP: kg.txt already processed
SKIP: sisdis.txt already processed
SKIP: ta-individu.txt already processed
SKIP: gamedev.txt already processed
SKIP: pbk.txt already processed
SKIP: kowan.txt already processed
SKIP: pbl.txt already processed
SKIP: pemlel.txt already processed
SKIP: kriptografi.txt already processed
SKIP: bioinformatika.txt already processed
SKIP: rpp.txt already processed
SKIP: aaw.txt already processed
SKIP: os.txt already processed
SKIP: ta-kelompok.txt already processed
SKIP: cv.txt already processed
SKIP: dl.txt already process

/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:364: UserWarning: Sentence of length 795 has been truncated to 768
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


DONE


In [ ]:
kb.persist_to_neo4j(
    uri="uri",
    username="neo4j",
    password="password"
)